In [243]:
import json
import csv
import pandas as pd
from datetime import datetime
from functools import reduce
from pandas.io.json import json_normalize

In [244]:
# Reading in our json files
with open('legacy_desktop_200712-201607.json', 'r') as json_file:
    legacy_desktop = json.loads(json_file.read())
    
with open('legacy_mobile_200712-201607.json', 'r') as json_file:
    legacy_mobile = json.loads(json_file.read())
    
with open('pageviews_website_201507-202109.json', 'r') as json_file:
    pageviews_website = json.loads(json_file.read())
    
with open('pageviews_desktop_201507-202109.json', 'r') as json_file:
    pageviews_desktop = json.loads(json_file.read())
    
with open('pageviews_mobile_201507-202109.json', 'r') as json_file:
    pageviews_mobile = json.loads(json_file.read())

In [245]:
# Creating 5 dataframes from each of our 5 json files

legacy_desktop_df = json_normalize(legacy_desktop, record_path = ['items'])
legacy_mobile_df = json_normalize(legacy_mobile, record_path = ['items'])
pageviews_website_df = json_normalize(pageviews_website, record_path = ['items'])
pageviews_desktop_df = json_normalize(pageviews_desktop, record_path = ['items'])
pageviews_mobile_df = json_normalize(pageviews_mobile, record_path = ['items'])

In [246]:
# combining the pageviews website and mobile dataframes by adding their viewcount by month
pageviews_allmobile_df = pd.concat([pageviews_website_df, 
                                    pageviews_mobile_df]).groupby(['timestamp'], as_index = False).sum().reset_index()


In [247]:
# Creating the Year and Month Columns for each of the 5 dataframes
def create_date_columns(df):
    df['Year'] = df.timestamp.apply(lambda x: x[:4])
    df['Month'] = df.timestamp.apply(lambda x: x[4:6])
    return df

legacy_desktop_df = create_date_columns(legacy_desktop_df)
legacy_mobile_df = create_date_columns(legacy_mobile_df)
pageviews_desktop_df = create_date_columns(pageviews_desktop_df)
pageviews_allmobile_df = create_date_columns(pageviews_allmobile_df)

# Renaming columns 
legacy_desktop_df.rename(columns={'count' : 'pagecount_desktop_views'}, inplace=True)
legacy_mobile_df.rename(columns={'count' : 'pagecount_mobile_views'}, inplace=True)
pageviews_desktop_df.rename(columns={'views' : 'pageviews_desktop_views'}, inplace=True)
pageviews_allmobile_df.rename(columns={'views' : 'pageviews_mobile_views'}, inplace=True)

# Dropping the 'Project', 'access-site', and 'Granularity' columns
legacy_desktop_df.drop(['project','access-site','granularity', 'timestamp'], axis=1, inplace=True)
legacy_mobile_df.drop(['project','access-site','granularity', 'timestamp'], axis=1, inplace=True)
pageviews_desktop_df.drop(['project','access', 'agent','granularity', 'timestamp'], axis=1, inplace=True)
pageviews_allmobile_df.drop(['index','timestamp'], axis=1, inplace=True)



In [248]:
# Creating 2 dataframes containing all the views from both the PageCount and PageView APIs
legacy_allviews_df = pd.concat([legacy_desktop_df, 
                                    legacy_mobile_df]).groupby(['Year','Month'], as_index = False).sum().reset_index()

pageviews_allviews_df = pd.concat([pageviews_desktop_df, 
                                    pageviews_allmobile_df]).groupby(['Year','Month'], as_index = False).sum().reset_index()

pageviews_allviews_df['pageviews_all_views'] = pageviews_allviews_df['pageviews_desktop_views'] + pageviews_allviews_df['pageviews_mobile_views']
legacy_allviews_df['pagecount_allviews_views'] = legacy_allviews_df['pagecount_desktop_views'] + legacy_allviews_df['pagecount_mobile_views']
pageviews_allviews_df.drop(['pageviews_desktop_views', 'pageviews_mobile_views'], axis=1, inplace=True)
legacy_allviews_df.drop(['pagecount_desktop_views', 'pagecount_mobile_views'], axis=1, inplace=True)

# making a list of our now 6 new dataframes
data_frames = [legacy_desktop_df, legacy_mobile_df, 
               pageviews_desktop_df, pageviews_allmobile_df, pageviews_allviews_df, legacy_allviews_df]

C:\Users\mdbla\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mdbla\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [249]:
all_views_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Year','Month'],
                                            how='outer'), data_frames).fillna(0)

all_views_merged.drop(['index_x', 'index_y'], axis=1, inplace=True)
all_views_merged

,pagecount_desktop_views,Year,Month,pagecount_mobile_views,pageviews_desktop_views,pageviews_mobile_views,pageviews_all_views,pagecount_allviews_views
0,2.998332e+09,2007,12,0.0,0.000000e+00,0.000000e+00,0.000000e+00,2.998332e+09
1,4.930903e+09,2008,01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,4.930903e+09
2,4.818394e+09,2008,02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,4.818394e+09
3,4.955406e+09,2008,03,0.0,0.000000e+00,0.000000e+00,0.000000e+00,4.955406e+09
4,5.159162e+09,2008,04,0.0,0.000000e+00,0.000000e+00,0.000000e+00,5.159162e+09
...,...,...,...,...,...,...,...,...
160,0.000000e+00,2021,04,0.0,2.882714e+09,4.759095e+09,7.641810e+09,0.000000e+00
161,0.000000e+00,2021,05,0.0,2.824416e+09,4.976580e+09,7.800996e+09,0.000000e+00
162,0.000000e+00,2021,06,0.0,2.505971e+09,4.584510e+09,7.090482e+09,0.000000e+00
163,0.000000e+00,2021,07,0.0,2.765584e+09,4.778909e+09,7.544494e+09,0.000000e+00


In [250]:
all_views_merged.to_csv('en-wikipedia_traffic_200712-202108.csv', index=False)